In [1]:
import sys
import os
import time
import threading
import utilities

from kortex_api.autogen.client_stubs.BaseClientRpc import BaseClient
from kortex_api.autogen.client_stubs.BaseCyclicClientRpc import BaseCyclicClient

from kortex_api.autogen.messages import Base_pb2, BaseCyclic_pb2, Common_pb2

# Maximum allowed waiting time during actions (in seconds)
TIMEOUT_DURATION = 20

In [2]:
def check_for_end_or_abort(e):
    """Return a closure checking for END or ABORT notifications

    Arguments:
    e -- event to signal when the action is completed
        (will be set when an END or ABORT occurs)
    """
    def check(notification, e = e):
        print("EVENT : " + \
              Base_pb2.ActionEvent.Name(notification.action_event))
        if notification.action_event == Base_pb2.ACTION_END \
        or notification.action_event == Base_pb2.ACTION_ABORT:
            e.set()
    return check


def example_move_to_home_position(base):
    # Make sure the arm is in Single Level Servoing mode
    base_servo_mode = Base_pb2.ServoingModeInformation()
    base_servo_mode.servoing_mode = Base_pb2.SINGLE_LEVEL_SERVOING
    base.SetServoingMode(base_servo_mode)
    
    # Move arm to ready position
    print("Moving the arm to home position")
    action_type = Base_pb2.RequestedActionType()
    action_type.action_type = Base_pb2.REACH_JOINT_ANGLES
    action_list = base.ReadAllActions(action_type)
    action_handle = None
    for action in action_list.action_list:
        if action.name == "Home":
            action_handle = action.handle

    if action_handle == None:
        print("Can't reach safe position. Exiting")
        return False

    e = threading.Event()
    notification_handle = base.OnNotificationActionTopic(
        check_for_end_or_abort(e),
        Base_pb2.NotificationOptions()
    )

    base.ExecuteActionFromReference(action_handle)
    finished = e.wait(TIMEOUT_DURATION)
    base.Unsubscribe(notification_handle)

    if finished:
        print("home position reached")
    else:
        print("Timeout on action notification wait")
    return finished

!!!!!!! ROBOT MOVING!!!!!!TO HOME!!!!!

In [12]:
class TCPArguments:
    def __init__(self):
        self.ip = "192.168.1.10"
        self.username = "admin"
        self.password = "admin"

args = TCPArguments()
# Create connection to the device and get the router
with utilities.DeviceConnection.createTcpConnection(args) as router:

    # Create required services
    base = BaseClient(router)
    base_cyclic = BaseCyclicClient(router)

    # Example core
    success = True

    success &= example_move_to_home_position(base)


Logging as admin on device 192.168.1.10
Moving the arm to a safe position
EVENT : ACTION_START
EVENT : ACTION_FEEDBACK
EVENT : ACTION_END
Safe position reached


In [1]:
from kortex_api.autogen.client_stubs.BaseClientRpc import BaseClient
from kortex_api.autogen.messages import Session_pb2, DeviceManager_pb2, Base_pb2
import time

def connect_to_robot():
    # Create a client instance
    with BaseClient() as client:
        try:
            client.CreateSession()
            print("Session created")

            # Connect to the first available robot
            print("Connecting to the first available robot...")
            client_device_manager = client.device_manager
            list_devices_info = client_device_manager.ReadAllDevices()
            for device_id in list_devices_info.device_handle:
                device_id = device_id.device_identifier
                print("Found device: ", device_id)
                client_device_manager.ConnectToDevice(device_id)
                print("Connected to device: ", device_id)
                break  # Connects only to the first available device
            else:
                print("No devices found.")

            yield client
        except Exception as e:
            print("Exception occurred: ", e)
            client.EndSession()

def move_end_effector(client, end_effector_pose):
    base = client.base
    twist_command = Base_pb2.TwistCommand()
    twist_command.twist.linear_x = 0.0  # adjust linear velocity
    twist_command.twist.linear_y = 0.0
    twist_command.twist.linear_z = 0.0
    twist_command.twist.angular_x = 0.0
    twist_command.twist.angular_y = 0.0
    twist_command.twist.angular_z = 0.0  # adjust angular velocity

    max_iterations = 100
    epsilon = 0.001  # convergence criterion
    for i in range(max_iterations):
        current_pose = base.GetActualEndEffectorPose().pose
        error = calculate_error(end_effector_pose, current_pose)
        if error < epsilon:
            print("Desired position reached.")
            break
        twist_command.twist.linear_x = error[0] * 0.1  # proportional control
        twist_command.twist.linear_y = error[1] * 0.1
        twist_command.twist.linear_z = error[2] * 0.1
        base.SendTwistCommand(twist_command)
        time.sleep(0.1)

def calculate_error(desired_pose, current_pose):
    # Implement your error calculation function here
    pass

def main():
    with connect_to_robot() as client:
        # Define the desired end-effector position
        desired_end_effector_pose = Base_pb2.Pose()
        desired_end_effector_pose.position.x = 0.1
        desired_end_effector_pose.position.y = 0.2
        desired_end_effector_pose.position.z = 0.3
        desired_end_effector_pose.orientation.x = 0.0
        desired_end_effector_pose.orientation.y = 0.0
        desired_end_effector_pose.orientation.z = 0.0
        desired_end_effector_pose.orientation.w = 1.0

        move_end_effector(client, desired_end_effector_pose)